In [26]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import Input, Dropout
from keras import optimizers

%matplotlib inline

In [27]:
train = pd.read_csv("sales_train_v2.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv('sample_submission.csv')
items = pd.read_csv('items.csv')
item_cats = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

In [29]:
#Peguei uma amostra pequena pra não demorar tanto
train = train.sample(frac=0.05, replace=False)

#Adicionando colunas de mês e ano para fazer agrupamento de compras por mês
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train = train.drop(['item_price','date'],1)

#Adicionando categoria do produto
sales = train.merge(items, left_on='item_id', right_on='item_id', how='inner')
sales = sales.drop('item_name',1)
sales.head(2)

In [31]:
#Agrupar vendas por mês, tirar a média e colocar no dataset principal

sales = train.groupby([c for c in train.columns if c not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
sales = sales.rename(columns={'item_cnt_day':'item_cnt_month'})

month_mean = sales[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False).mean()
month_mean = month_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})

sales = pd.merge(sales, month_mean, how='left', on=['shop_id','item_id'])
sales.head()

In [34]:
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34
#Add Mean Feature
test = pd.merge(test, month_mean, how='left', on=['shop_id','item_id']).fillna(0.)
#Items features
test['item_cnt_month'] = 0.

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_month
0,0,5,5037,11,2015,34,0.0,0.0
1,1,5,5320,11,2015,34,0.0,0.0
2,2,5,5233,11,2015,34,1.0,0.0
3,3,5,5232,11,2015,34,0.0,0.0
4,4,5,5268,11,2015,34,0.0,0.0


In [35]:
test = test[['date_block_num', 'shop_id', 'item_id', 'year', 'month','item_cnt_month', 'item_cnt_month_mean']]
test.head()

,date_block_num,shop_id,item_id,year,month,item_cnt_month,item_cnt_month_mean
0,34,5,5037,2015,11,0.0,0.0
1,34,5,5320,2015,11,0.0,0.0
2,34,5,5233,2015,11,0.0,1.0
3,34,5,5232,2015,11,0.0,0.0
4,34,5,5268,2015,11,0.0,0.0


In [55]:
#Colocar os dados de treino na mesma escala para performance do modelo
dados_valores = sales.values

X = dados_valores[:,0:6]
Y = dados_valores[:,6]

sc = StandardScaler()
X_scaled = sc.fit_transform(X)

# Conjuntos de dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = model_selection.train_test_split(X_scaled, Y, test_size = 0.30, random_state = 7)

In [56]:
def linear():
    model = Sequential()
    
    model.add(Dense(4, input_dim=6, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='adam')
    return model

In [57]:
model = linear()
model.fit(X_treino, Y_treino, nb_epoch=20, batch_size=256,verbose=1)

C:\Users\mathe\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/20
99916/99916 [==============================] - 5s 50us/step - loss: 4.1543
Epoch 2/20
99916/99916 [==============================] - 4s 43us/step - loss: 3.1159
Epoch 3/20
99916/99916 [==============================] - 4s 43us/step - loss: 2.5467
Epoch 4/20
99916/99916 [==============================] - 5s 46us/step - loss: 2.1942
Epoch 5/20
99916/99916 [==============================] - 4s 44us/step - loss: 1.8555
Epoch 6/20
99916/99916 [==============================] - 4s 43us/step - loss: 1.6843
Epoch 7/20
99916/99916 [==============================] - 4s 40us/step - loss: 1.6414
Epoch 8/20
99916/99916 [==============================] - 5s 45us/step - loss: 1.6264
Epoch 9/20
99916/99916 [==============================] - 4s 42us/step - loss: 1.6189
Epoch 10/20
99916/99916 [==============================] - 4s 43us/step - loss: 1.6209
Epoch 11/20
99916/99916 [==============================] - 4s 40us/step - loss: 1.6176
Epoch 12/20
99916/99916 [===========================

In [60]:
ynew = model.predict(X_teste)
print("X=%s, Predicted=%s" % (X_teste[0], ynew[0]))

X=[ 1.42545429  1.23580897 -1.37897524  1.59120622 -0.40288849 -0.10846762], Predicted=[1.0704131]
